In [101]:
# 필요한 라이브러리 임포트
import requests
import re
import time, random
import pandas as pd
from lxml import html
from IPython.display import Image, display

In [90]:
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [92]:
# csv로 파일 목록 읽어와 데이터프레임화하기
df = pd.read_csv('thy_index.csv')
df.head()

,date,code
0,1883-01-01,sa_024r_0010_0010_0010
1,1883-01-02,sa_024r_0010_0010_0020
2,1883-01-03,sa_024r_0010_0010_0030
3,1883-01-04,sa_024r_0010_0010_0040
4,1883-01-05,sa_024r_0010_0010_0050


In [97]:
df.code.size

2283

In [100]:
# 크롤링 성공한 요청 예시
cookies = {
    'activexime_enable': 'false',
    'PCID': '17056246878505931577813',
    'RC_RESOLUTION': '1536*864',
    'RC_COLOR': '24',
    'activexime_enable': 'false',
    'JSESSIONID': 'D67983200082058E074FA74054AD0BC5',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    # 'Cookie': 'activexime_enable=false; PCID=17056246878505931577813; RC_RESOLUTION=1536*864; RC_COLOR=24; activexime_enable=false; JSESSIONID=D67983200082058E074FA74054AD0BC5',
    # 'Referer': f'https://db.history.go.kr/item/compareViewer.do?levelId={df.code[0]}',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

params = {
    # 'levelId': df.code[0],
}


In [104]:
# for문 돌려서 읽어오고 바로 데이터프레임 별도의 열에 저장하기
diary_entries = []

for i in range(df.code.size):
    headers['Referer'] = f'https://db.history.go.kr/item/compareViewer.do?levelId={df.code[i]}'
    params['levelId'] = df.code[i]
    response = requests.get('https://db.history.go.kr/item/compareViewer.do', params=params, cookies=cookies, headers=headers, verify=False)

    temp = html.fromstring(response.content).xpath('//*[@id="rightContent"]//div//text()')
    tokens = [s.strip() for s in temp if '\n' not in s and s != ' ' and s != '닫기']
    proc_str = ' '.join(tokens)
    proc_str = proc_str.replace(" )", ")").replace("( ", "(").replace("“ ", "“").replace(" ”", "”").replace("원저자주 ", "").replace("선⃐", "")
    # re.compile(r'[一-龥]').sub('', proc_str)
    # re.compile(r'\([^)]*\)').sub('', proc_str)
    diary_entries.append(proc_str)
    time.sleep(random.randrange(1,5))

print('크롤링 완료')
df['diary_text'] = pd.Series(diary_entries)
df.head()

크롤링 완료


,date,code,diary_text
0,1883-01-01,sa_024r_0010_0010_0010,"1883년(명치 16) 1월 1일(1882년 11월 22일, 월, 맑고 춥다) 관직..."
1,1883-01-02,sa_024r_0010_0010_0020,"2일(23일, 화, 맑음) 고우장(古愚丈: 김옥균)과 함께 후쿠자와 유키치(福澤諭吉..."
2,1883-01-03,sa_024r_0010_0010_0030,"3일(24일, 수, 맑고 춥다) 이날 오전에 고우장(古愚丈: 김옥균)과 같이 가미즈..."
3,1883-01-04,sa_024r_0010_0010_0040,"4일(25일, 목, 맑고 춥다) 아침 10시 기차로 고우(古愚: 김옥균), 각치공(..."
4,1883-01-05,sa_024r_0010_0010_0050,"5일(26일, 금, 맑고 춥다) 아침에 북해(北海:손붕구 孫鵬九) 씨 댁에 가서 종..."


In [127]:
df[df['diary_text'].str.contains(r"[0-9]{3}\s[0-9]{3}", na=False)].date

0       1883-01-01
2       1883-01-03
12      1883-01-13
16      1883-10-19
19      1883-10-22
           ...    
2274    1889-12-23
2275    1889-12-24
2278    1889-12-27
2279    1889-12-28
2280    1889-12-29
Name: date, Length: 65, dtype: object

In [108]:
# df['text_without_CN'] = df['diary_text'].str.replace(pat=r'\([^)]*\)', repl='', regex=True)
# "이 기사는 국사편찬위원회에서 간행한 국역 윤치호 영문 일기1에 수록된 내용입니다." 문자열을 포함하는지도 점검해야 함 

date          0
code          0
diary_text    0
dtype: int64

In [113]:
# csv로 다시 저장하기
df.to_csv('thy_diary.csv', index=False, encoding='utf-8')